In [1]:
import numpy as np
import pandas as pd


In [2]:
from gensim.models.fasttext import FastText


In [3]:
model = FastText.load_fasttext_format('../data/cc.ru.300.bin')
print(model.wv['мир'])

/var/folders/jq/byk1x73j6pngqgl6my8tgyhm0000gn/T/ipykernel_10267/3231241568.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  model = FastText.load_fasttext_format('../data/cc.ru.300.bin')


[-0.04010157 -0.01939676  0.0123472   0.00420404 -0.14823544 -0.14018615
  0.10929612 -0.07573465  0.09767959  0.21756363 -0.07176124  0.08621705
  0.15654308  0.03002805 -0.0368893   0.05824615  0.11482157  0.02317262
  0.04539815 -0.0355209  -0.03603294 -0.1221268  -0.06845482  0.04976111
  0.04596197  0.04964927  0.01915357  0.04100516  0.01401335 -0.08294429
  0.16267993  0.06430586  0.01752418 -0.06838899  0.06295058 -0.04523483
 -0.13037588 -0.2482847   0.01887992 -0.16284476 -0.09788799  0.06642605
 -0.10626589  0.04866524  0.04178581 -0.02333352 -0.2725895   0.18531033
  0.19238655  0.13830474  0.11559307  0.13343623  0.22933286 -0.11461251
  0.14354116  0.03846733  0.04947425 -0.07110243 -0.01087112  0.08804761
 -0.06343422  0.08565489 -0.15966357 -0.07254395  0.1130814  -0.00968511
  0.04944057 -0.16401084 -0.03669032 -0.00898871 -0.10663027 -0.02158327
  0.08737282 -0.01769257 -0.03525456  0.08112386 -0.17629416  0.07922973
  0.08331937 -0.06936362 -0.14728579 -0.16582023 -0

In [4]:
def normalize_vector(vec):
    norm = np.sqrt(np.sum(vec**2))
    if not norm==0:
        return vec/norm
    else:
        return vec


In [5]:
df = pd.read_csv('../data/data_vacancies.csv')


In [6]:
def get_sentence_vector(s: str) -> np.ndarray:
    return np.mean([model.wv[s_i] for s_i in s.split()
    ], axis=0
    )
df['emb'] = df.custom_position.apply(lambda x: get_sentence_vector(x))


In [7]:
df['salary_mean'] = (df.salary_from + df.salary_to) / 2
df.salary_mean.head()


0    90000.0
1    90000.0
2    70000.0
3    32500.0
4    32500.0
Name: salary_mean, dtype: float64

In [40]:
df = df[df.salary_mean < df.salary_mean.quantile(0.95)]


In [41]:
X, y = df[['emb']], df.salary_mean

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(np.array([np.array(x) for x in X.emb.to_numpy()]), y, test_size=0.2, random_state=42)

model = LinearRegression()


In [43]:
model.fit(X_train, y_train)


LinearRegression()

In [46]:
from sklearn.ensemble import GradientBoostingRegressor


In [44]:
y_pred = model.predict(X_test)

mse = mean_absolute_error(y_test, y_pred)

print('The mean squared error is ', mse)

The mean squared error is  16277.157743659482


In [47]:
model = GradientBoostingRegressor()
model.fit(X_train, y_train)


GradientBoostingRegressor()

In [48]:
y_pred = model.predict(X_test)

mse = mean_absolute_error(y_test, y_pred)

print('The mean squared error is ', mse)


The mean squared error is  15069.370709675597
